# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv 
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Iqaluit,63.7506,-68.5145,8.85,76,40,2.57,CA,1720631878
1,1,Wailua Homesteads,22.0669,-159.3780,24.44,86,0,7.20,US,1720631878
2,2,Atafu Village,-8.5421,-172.5159,28.73,69,95,7.12,TK,1720631878
3,3,Manokwari,-0.8667,134.0833,25.04,83,75,1.91,ID,1720631878
4,4,Xining,36.6167,101.7667,15.50,80,75,0.47,CN,1720631878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_height = 500,
    frame_width = 700,
    alpha = 0.5,
    color = "City",
    scale = 1
)

map_plot_1

# Display the map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
optimal_weather_df = city_data_df[(city_data_df["Max Temperature"] > 20) & (city_data_df["Max Temperature"] < 28) & 
             (city_data_df["Humidity"] < 30)].dropna()

# Drop any rows with null values
# Display sample data
optimal_weather_df

,City_ID,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
114,114,Kumul,42.8000,93.4500,27.90,28,0,6.93,CN,1720631893
131,131,Chelak,39.9203,66.8611,27.42,21,86,5.07,UZ,1720631895
150,150,Norak,38.3892,69.3227,27.84,24,3,2.40,TJ,1720631898
275,275,Tabora,-5.0167,32.8000,22.99,28,81,2.24,TZ,1720631916
494,494,São Vicente,-23.9631,-46.3919,26.10,26,73,2.82,BR,1720631953
518,518,Baynjiwayn,35.6183,45.9541,27.79,16,33,1.63,IQ,1720631956
540,540,Saurimo,-9.6608,20.3916,21.94,14,46,1.91,AO,1720631959
580,580,Patterson,37.4716,-121.1297,27.08,25,0,2.55,US,1720631966
585,585,Ilebo,-4.3167,20.5833,27.32,20,97,0.67,CD,1720631966


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame(optimal_weather_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temperature'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temperature,Hotel Name
114,Kumul,CN,42.8000,93.4500,27.90,
131,Chelak,UZ,39.9203,66.8611,27.42,
150,Norak,TJ,38.3892,69.3227,27.84,
275,Tabora,TZ,-5.0167,32.8000,22.99,
494,São Vicente,BR,-23.9631,-46.3919,26.10,
518,Baynjiwayn,IQ,35.6183,45.9541,27.79,
540,Saurimo,AO,-9.6608,20.3916,21.94,
580,Patterson,US,37.4716,-121.1297,27.08,
585,Ilebo,CD,-4.3167,20.5833,27.32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { "apiKey":geoapify_key,
          "categories":'accommodation.hotel	',
          "limit": 20,
          
          
          
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
          
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    

    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    hotel_df = hotel_df[hotel_df['Hotel Name']!= 'No hotel found']

# Display sample data
hotel_df

Starting hotel search
Kumul - nearest hotel: Хами
Chelak - nearest hotel: No hotel found
Norak - nearest hotel: Sayora.S Hotel
Tabora - nearest hotel: Alpho hotel
São Vicente - nearest hotel: Pousada Vitória
Baynjiwayn - nearest hotel: No hotel found
Saurimo - nearest hotel: Hotel Princesinha
Patterson - nearest hotel: Hampton Inn & Suites
Ilebo - nearest hotel: Hôtel des Palmes


,City,Country,Lat,Lng,Max Temperature,Hotel Name
114,Kumul,CN,42.8000,93.4500,27.90,Хами
150,Norak,TJ,38.3892,69.3227,27.84,Sayora.S Hotel
275,Tabora,TZ,-5.0167,32.8000,22.99,Alpho hotel
494,São Vicente,BR,-23.9631,-46.3919,26.10,Pousada Vitória
540,Saurimo,AO,-9.6608,20.3916,21.94,Hotel Princesinha
580,Patterson,US,37.4716,-121.1297,27.08,Hampton Inn & Suites
585,Ilebo,CD,-4.3167,20.5833,27.32,Hôtel des Palmes


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_height = 500,
    frame_width = 700,
    alpha = 0.9,
    color = "City",
    scale = 1,
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)